In [4]:
import numpy as np
import numpy.random as rnd
import scipy
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from numpy import linalg as LA
import cvxpy as cp
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [5]:
from exp_setup import Experiment

In [6]:
from algorithms import *

In [7]:
num_species = 4 #drone,rover,mini-rover,mini-drone
num_tasks = 3  #pick,search for target,move object
num_traits = 9 #speed,footprint,payload,reach,weight,sensing frequency,sensing range,color,battery capacity
traits = ["speed","footprint","payload","reach","weight","sensing frequency","sensing range","color","battery"]
num_demo = 1000

In [8]:
exp = Experiment()
D = exp.get_expert_demonstrations()

In [115]:
new_Q = exp.get_random_q()
optimal_Y = exp.get_optimal_Y()
agents_target = exp.get_agent_limit()
Q = D["Q"]
Y = D["Y"]
Y_mean = np.mean(Y, axis=0)
new_Q

array([[  8.66002208,  89.87356781,   8.32657535,   1.11629356,
         25.95750418, 183.5444571 ,  59.01794731,   0.        ,
         12.        ],
       [  6.23672893, 136.34302107,   3.8647693 ,   1.5398084 ,
         26.25056173, 234.2447839 ,  60.10517307,   1.        ,
         12.        ],
       [  6.72051716, 107.69444947,  16.27204719,   1.95470247,
         26.99182765, 261.09833978,  59.49202801,   4.        ,
         11.        ],
       [ 16.84758602,  91.80880827,  11.02789632,   3.84383249,
         26.01533541, 209.16171698,  58.8684424 ,   1.        ,
         13.        ]])

In [116]:
X_mat = solve_task_allocation(agents_target,new_Q,Y_mean)
print("mat" , X_mat)

X_per_task = solve_task_allocation_per_task(agents_target,new_Q,Y_mean)
print("task based" , X_per_task)

mat [[0. 0. 4. 7.]
 [4. 5. 3. 1.]
 [1. 3. 0. 0.]]
task based [array([-0., -0.,  4.,  7.]), array([4., 5., 3., 1.]), array([ 1.,  3., -0., -0.])]


In [117]:
Y_mat = X_mat@new_Q
print(LA.norm(optimal_Y-Y_mat, 2)/LA.norm(optimal_Y,2))

Y_per_task = X_per_task@new_Q
print(LA.norm(optimal_Y-Y_per_task, 2)/LA.norm(optimal_Y,2))


0.392649698918157
0.392649698918157


Baseline : Weighted optimization

In [118]:
Q_D = np.concatenate(Q)
var_N = np.var(Q_D,axis=0)
var_O = np.var(Y,axis=0)
print(var_O)
print(var_N)

[[ 542.19574868 7352.11974886  454.78414421  404.22988833  660.02095334
  6011.06031257 1645.93733022  196.0255839   877.0969751 ]
 [ 224.30108478 1156.26341011  766.19473029   97.64859299  649.71067912
  3197.9249996  1220.76916463  166.062636    778.54375658]
 [  25.28578288 4314.70272239  172.62686286   17.01016715  148.63996863
  3219.62047118  466.85288685   30.82281603  150.84150377]]
[1.61284264e+01 1.13233642e+03 2.00037653e+01 2.02320772e+00
 3.01499114e+00 2.77705823e+02 9.06265597e-01 1.98499100e+00
 8.13375900e+00]


In [163]:
cv_N = np.sqrt(var_N)/np.mean(Q_D,axis=0)
cv_O = np.sqrt(var_O)/Y_mean

norm_var_N = cv_N/np.sum(cv_N)
norm_var_O = cv_O
for m in range(num_tasks):
    norm_var_O[m] /= np.sum(cv_O[m])

print(norm_var_O)
print(norm_var_N)

# print(np.sum(norm_var_O,axis=1))
# print(np.sum(norm_var_N))

[[0.06882699 0.05156232 0.12437458 0.29508669 0.0399809  0.01433228
  0.02905381 0.29182208 0.08496035]
 [0.0902492  0.01561608 0.16798976 0.21326497 0.04796768 0.01310454
  0.03021464 0.32468382 0.09690931]
 [0.10069277 0.04467602 0.2255371  0.14520248 0.04670924 0.02796126
  0.03808487 0.28481125 0.086325  ]]
[0.1516066  0.10952802 0.17331303 0.19689246 0.02252941 0.02649311
 0.00568603 0.24563681 0.06831452]


In [120]:
def calculate_weight(natural_variance, observerd_variance):
    weights = []
    for i in range(num_traits):
        weights.append((natural_variance[i])*math.cos(2*observerd_variance[i]) + 0.5)
    
    return weights

In [129]:
w = [calculate_weight(norm_var_N,norm_var_O[m]) for m in range(num_tasks)]
print(w)
sum_w = np.sum(w,axis=1)

for m in range(num_tasks):
    w[m] /= sum_w[m]

# print(w)

[[0.6501724942776326, 0.6089461358550492, 0.6679786520066902, 0.6635869623316206, 0.5224574236367432, 0.5264822296829017, 0.5056764373974132, 0.7049740955673216, 0.567330669454948], [0.6491436464393557, 0.6094746031781509, 0.6636227231086642, 0.679252245837195, 0.522425814287802, 0.5264840143137335, 0.5056756554476783, 0.6956414494760359, 0.5670353947986158], [0.6485426825656173, 0.6090910848376068, 0.6559781483865639, 0.6886481831628019, 0.5224311747811942, 0.5264516975282775, 0.5056695473987821, 0.7068518936823192, 0.5672988890859132]]


In [122]:
X = []
for m in range(num_tasks):
        X_sol = cp.Variable((num_species), integer=True)

        # minimize trait mismatch
        mismatch_mat = w[m]*Y_mean[m] - cp.multiply(w[m],cp.matmul(new_Q.T,X_sol))  # trait mismatch matrix
        # print(mismatch_mat.shape)
        # weighted_mat = cp.matmul(mismatch_mat, cp.matmul(W[m],mismatch_mat.T))
        # print(w[m],mismatch_mat)
        obj = cp.Minimize(cp.norm2(mismatch_mat))
        # ensure each agent is only assigned to one task
        constraints = [X_sol <= np.array(agents_target), X_sol >= 0]

        # solve for X_target
        opt_prob = cp.Problem(obj, constraints)
        opt_prob.solve(solver=cp.CPLEX)
        X_target = X_sol.value
        X.append(np.round(X_target))
X

[array([ 0., -0.,  4.,  7.]), array([4., 5., 3., 1.]), array([0., 4., 0., 0.])]

In [123]:
Y_baseline1 = X@new_Q
LA.norm(optimal_Y-Y_baseline1, 2)/LA.norm(optimal_Y,2)

0.3911602050336385

Baseline : Weighted optimization - Reduced Dimensions

In [160]:
w_new = [calculate_weight(norm_var_N,norm_var_O[m]) for m in range(num_tasks)]
for m in range(num_tasks):
    idx = np.argpartition(w[m], 3)
    print(idx)
    for i in range(1):
        w_new[m][idx[i]] = 0
sum_w_new = np.sum(w_new,axis=1)

for m in range(num_tasks):
    w_new[m] /= sum_w_new[m]

print(w_new)

[5 4 6 8 1 3 2 7 0]
[5 4 6 8 1 3 2 7 0]
[5 4 6 8 1 3 2 7 0]
[array([0.13292909, 0.12450027, 0.13656959, 0.1356717 , 0.10681748,
       0.        , 0.10338657, 0.14413338, 0.11599191]), array([0.13268758, 0.12457906, 0.13564716, 0.13884189, 0.10678594,
       0.        , 0.10336214, 0.14219191, 0.11590432]), array([0.1322339 , 0.12418996, 0.13374994, 0.14041116, 0.10652053,
       0.        , 0.10310294, 0.14412279, 0.11566878])]


In [161]:
X_baseline2 = []
for m in range(num_tasks):
        X_sol = cp.Variable((num_species), integer=True)
        # minimize trait mismatch
        mismatch_mat = w_new[m]*Y_mean[m] - cp.matmul(cp.multiply(np.tile(w_new[m],(num_species,1)),new_Q).T,X_sol)  # trait mismatch matrix
        # print(mismatch_mat)
        # weighted_mat = cp.matmul(mismatch_mat, cp.matmul(W[m],mismatch_mat.T))
        # print(w[m],mismatch_mat)
        obj = cp.Minimize(cp.norm2(mismatch_mat))
        # ensure each agent is only assigned to one task
        constraints = [X_sol <= np.array(agents_target), X_sol >= 0]

        # solve for X_target
        opt_prob = cp.Problem(obj, constraints)
        opt_prob.solve(solver=cp.CPLEX)
        X_target = X_sol.value
        X_baseline2.append(np.round(X_target))
X_baseline2

[array([ 3., -0., -0.,  7.]), array([3., 5., 3., 2.]), array([0., 4., 0., 0.])]

In [162]:
Y_baseline2 = X_baseline2@new_Q
LA.norm(optimal_Y-Y_baseline2, 2)/LA.norm(optimal_Y,2)

0.4166349407855685